In [2]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.0 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=d25eef1e38ac9e8d11e3ca0ec071f321a9ae9cc6a57f6284cb10c8eb912a6983
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [3]:
! pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 90.0 MB/s eta 0:00:00


In [4]:
!unzip /content/final_train_dataset.csv.zip

Archive:  /content/final_train_dataset.csv.zip
  inflating: final_train_dataset (1).csv  
  inflating: __MACOSX/._final_train_dataset (1).csv  


In [5]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
from tqdm.notebook import tqdm

#### Getting data

In [6]:
df = pd.read_csv('/content/final_train_dataset (1).csv', on_bad_lines = 'warn')
#df['context'] = df['context'].fillna('')
df['context'].isna().sum()

1271

In [7]:
df['context'] = df['context'].fillna('')

In [8]:
def get_documents(data: pd.DataFrame) -> list:
    result = []
    answers = data['context'].to_list()
    for i in range(len(answers)):
        try:
          #print(answers[i])
          result.append(answers[i])
        # except Exception as e:
        #   continue
        except SyntaxError as e:
          continue
        except NameError as e:
          continue

    return result

In [9]:
answers = list(get_documents(df))

In [10]:

answers = set(answers)

In [11]:
long_answers = []
for i in answers:
    if len(i) > 50:
        long_answers.append(i)

In [12]:
len(long_answers)

59866

In [ ]:
questions = df['question'].to_list()

In [13]:
document_mapper = dict(zip(list(range(len(long_answers))), long_answers))

#### Creating index

In [19]:
class Embedding_model:
    def __init__(self):
        self.transformer = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased', device=f"cuda:0")

    def __call__(self, text_batch):
        embeddings = self.transformer.encode(
            text_batch,
            batch_size=100,
            device=f"cuda:0",
        )

        return embeddings

In [20]:
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):

  def __init__(self, document_mapper: dict):
    self.answers = list(document_mapper.values())
    self.indexes = list(document_mapper.keys())

  def __len__(self):
    return len(self.answers)

  def __getitem__(self, idx):
    return self.answers[idx], self.indexes[idx]

In [21]:
def creating_index(document_mapper):
    dataset = MyDataset(document_mapper)
    data_loader = DataLoader(dataset, batch_size=1000, shuffle=True)

    model = Embedding_model()

    base_index = faiss.IndexFlat(512)

    index = faiss.IndexIDMap2(base_index)

    for question, ids in tqdm(data_loader):
        vectors = model(question)

        index.add_with_ids(vectors, ids)

    faiss.write_index(index, f'faiss_index')

    return index

In [28]:
index = creating_index(document_mapper)

  0%|          | 0/60 [00:00<?, ?it/s]

#### Retrieving

In [16]:
!unzip faiss_index.zip

Archive:  faiss_index.zip
  inflating: faiss_index (1)         
  inflating: __MACOSX/._faiss_index (1)  


In [17]:
#index = faiss.read_index('faiss_index (1)')

In [29]:
model = Embedding_model()

In [30]:
from sentence_transformers.util import dot_score

def mmr(query_embedding: np.ndarray,
        reviews_embeddings: np.ndarray,
        reviews,
        diversity: float = 0.1,
        top_n: int = 10):
    """ Maximal Marginal Relevance
    Arguments:
        query_embedding: The document embeddings
        reviews_embeddings: The embeddings of the selected candidate keywords/phrases
        reviews: The selected candidate keywords/keyphrases
        diversity: The diversity of the selected embeddings.
                   Values between 0 and 1.
        top_n: The top n items to return
    Returns:
            List[str]: The selected keywords/keyphrases
    """


    reviews_query_similarity = dot_score(reviews_embeddings, query_embedding).detach().numpy()
    reviews_similarity = np.dot(reviews_embeddings, reviews_embeddings.T)


    keywords_idx = [np.argmax(reviews_query_similarity)]
    mmr_ranks = [np.max(reviews_query_similarity)]

    candidates_idx = [i for i in range(len(reviews)) if i != keywords_idx[0]]

    for _ in range(top_n - 1):
        candidate_similarities = reviews_query_similarity[candidates_idx, :]
        target_similarities = np.max(reviews_similarity[candidates_idx][:, keywords_idx], axis=1)


        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_value = np.max(mmr)
        mmr_idx = candidates_idx[np.argmax(mmr)]


        keywords_idx.append(mmr_idx)
        mmr_ranks.append(mmr_value)
        candidates_idx.remove(mmr_idx)

    output_reviews = {}
    for i in range(len(keywords_idx)):
        text = reviews[keywords_idx[i]]
        output_reviews[text] = mmr_ranks[i]

    return output_reviews


In [31]:
def search(question, k_index=100, k_mmr=10, diversity=0.1):
    result_dict = {}

    query_emb = np.array([model(question)])
    D, I = index.search(query_emb, k=k_index)
    D, I = list(D[0]), list(I[0])

    vectors = []
    for i in range(len(I)):
        result_dict[document_mapper[I[i]]] = D[i]

    vectors = index.reconstruct_batch(I)
    mmr_dict = mmr(query_emb, vectors, list(result_dict.keys()), diversity, k_mmr)

    return result_dict , mmr_dict

In [32]:
def retrieve(answers: dict, mapper: dict):
  keys = list(answers.keys())
  if keys[0] <= 0.1:
    threshold = keys[0]
    relevants = [answers[key] for key in keys if key < threshold+0.08]
  else:
    relevents = keys[:4]

  ids = []
  for i in relevants:
    ids.append(list(document_mapper.keys())[list(document_mapper.values()).index(i)])

  return list(set(ids))


In [45]:
def nicely_retrieved(question):
  phrases = search(question)[1]
  top_3 = list(phrases.keys())[:3]
  return ';'.join(top_3)

In [51]:
print(*nicely_retrieved('Как справиться с депрессией?').split(';'), sep='\n\n')

Здравствуйте похоже на маниакально - депрессивное состояние, может быть как форма болезни, а  может как состояние. Ощущаете ли в периоды  апатии депрессию? Сколько длиться по времени состояние апатии ? 

ну мне кто то говорил что депрессия это грех вообще,тебе надо просто отдохнуть Смотря по каким причинам у тебя депрессия ты хочешь об этом поговорить ? найди дело по душе Ты просишь помощи, при этом не говоришь... от чего тебе помочь... откуда я могу знать... почему ты в депрессии.. по какой причине.... А судя по вопросу... твоя депрессия называется ЛЕНЬ! От нервов седативное а дальше работать над этим и большое желание вылезти. Без подробностей у вас не будет конкретных советов возможно хороший психолог нужен. Ещё возможно что вы депрессией какую нибудь фигню называете. Поговорить со своей головой. Взять листок с ручкой и записать все что тревожит, что бы хотелось изменить, что лишнее в жизни и тд.... а дальше осознанно стремиться к достижению спокойствия в душе... Нам не всегда под с

In [ ]:
! pip install sumy

In [74]:
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.nlp.stemmers import Stemmer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer

import nltk
nltk.download('punkt')

def summary(text):
  LANGUAGE = 'russian'
  parser = PlaintextParser.from_string(text, Tokenizer(LANGUAGE))
  stemmer = Stemmer(LANGUAGE)

  summarizer = Summarizer(stemmer)

  final =[]
  for sentence in summarizer(parser.document, 35):
    final.append(str(sentence))
  return ';'.join(final)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [54]:
print(nicely_retrieved('Что делать, если меня бросил парень?'))

А кто кого бросил? дружба с бывшим парнем невозможна. почему ты предложила расстаться? Чё делать? Спроси у лучшей подруги. Обычно они помогают избавляться от надоевших парней. Нужно помогать друг другу.;О смысле жизни ...) Куда я попаду?Куда надо?И когда? О деньгах О свечке.;уйду Отпущу его с миром. Не навязываюсь.Плохой тон. Ушёл же.. Ушла бы... Я выгоняю её из своей квартиры. Если люблю, то уйду.


In [76]:
type(summary(nicely_retrieved('Как справиться с депрессией?')))

str

In [84]:
with open('test_fin.txt', 'w') as f:
  questions = ['Как справиться с депрессией?', 'Как перестать грустить?',
               'Есть ли дружба между мужчиной и женщиной?', 'Как понять, что мои родители абьюзеры?', 'Как манипулировать парнем?']
  for i in questions:
    s_r = summary(nicely_retrieved(i))
    f.write(f'{i}  |  {s_r}\n\n')
    print(s_r)

Здравствуйте похоже на маниакально - депрессивное состояние, может быть как форма болезни, а  может как состояние.;Ощущаете ли в периоды  апатии депрессию?;Сколько длиться по времени состояние апатии ?;;ну мне кто то говорил что депрессия это грех вообще,тебе надо просто отдохнуть Смотря по каким причинам у тебя депрессия ты хочешь об этом поговорить ?;найди дело по душе Ты просишь помощи, при этом не говоришь... от чего тебе помочь... откуда я могу знать... почему ты в депрессии.. по какой причине.... А судя по вопросу... твоя депрессия называется ЛЕНЬ!;От нервов седативное а дальше работать над этим и большое желание вылезти.;Без подробностей у вас не будет конкретных советов возможно хороший психолог нужен.;Ещё возможно что вы депрессией какую нибудь фигню называете.;Взять листок с ручкой и записать все что тревожит, что бы хотелось изменить, что лишнее в жизни и тд.... а дальше осознанно стремиться к достижению спокойствия в душе... Нам не всегда под силу изменить свои обстоятельст